<p><font size="6" color='grey'> <b>
Generative KI
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
Retrieval-Augmented Generation (RAG) Chatbot für SQL-Datenbanken
</b></font> </br></p>

---



# **1 <font color='orange'>|</font> Setup und Installation**
---

In [ ]:
%%writefile requirements.txt
langchain>=0.2.0
langchain-experimental>=0.0.49
langchain-groq>=0.1.0
httpx==0.27.2
sqlalchemy>=2.0.0
gradio>=3.50.0
pydantic>=2.0.0

Installation der Pakete:

In [ ]:
!pip install -q -U -r requirements.txt

Module importieren:

In [ ]:
# Standard Libraries
import os
import sqlite3
from typing import Any

# Third Party Libraries
import gradio as gr
from google.colab import userdata
from langchain_experimental.sql.base import SQLDatabase, SQLDatabaseChain
from langchain_groq import ChatGroq
from pydantic import BaseModel

# **2 <font color='orange'>|</font> Konfiguration und Umgebungsvariablen**
---

In [ ]:
# Groq API Konfiguration
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# LLM Konfiguration
MODEL_NAME = "mixtral-8x7b-32768"  # Groq's Mixtral Modell
TEMPERATURE = 0

# Datenbank Konfiguration
DB_PATH = "/content/chinook.db"
DB_URI = f"sqlite:///{DB_PATH}"

# App Konfiguration
APP_TITLE = "📚 Datenbank Groq Chatbot"
APP_DESCRIPTION = "\n\n*Der Chatbot wertet die Datenbank Dokumente aus und beantwortet Fragen zum Inhalt*"

# LLM initialisieren
llm = ChatGroq(temperature=TEMPERATURE, model_name=MODEL_NAME)

# **3 <font color='orange'>|</font> Datenvorverarbeitung und Embedding**
---

Daten laden:

In [ ]:
!curl -L https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02%20data/chinook.db -o chinook.db

# **4 <font color='orange'>|</font> Memory und Kontextmanagement**
---

Datenbankkontext laden:

In [ ]:
def get_table_info(db_path: str) -> dict:
    """Extrahiert Schema-Informationen aus der SQLite Datenbank.

    Args:
        db_path: Pfad zur SQLite Datenbank

    Returns:
        Dictionary mit Tabellen und deren Spalten
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Tabellennamen abfragen
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in cursor.fetchall()]

    # Spalten für jede Tabelle abfragen
    table_info = {}
    for table_name in tables:
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = [column[1] for column in cursor.fetchall()]
        table_info[table_name] = columns

    conn.close()
    return table_info

# Schema-Informationen laden
table_info = {}
table_info = get_table_info(DB_PATH)

# **5 <font color='orange'>|</font> Prompt-Engineering**
---

SQLDatabaseChain Setup:

In [ ]:
# Workaround für Callback-Fehler
class Callbacks(BaseModel):
    pass
BaseCache = Callbacks
SQLDatabaseChain.model_rebuild()

# Datenbank-Chain initialisieren
db = SQLDatabase.from_uri(DB_URI)
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

# **6 <font color='orange'>|</font> RAG-Chatbot-Architektur**
---

In [ ]:
def run_query_with_schema_check(query: str,
                              db_chain=db_chain,
                              db_path=DB_PATH,
                              table_info=table_info) -> str:
    """Führt eine Benutzeranfrage unter Berücksichtigung des DB-Schemas aus.

    Args:
        query: Natürlichsprachliche Benutzeranfrage
        db_chain: LangChain SQLDatabaseChain
        db_path: Pfad zur Datenbank
        table_info: Schema-Informationen

    Returns:
        Antwort als String
    """
    # Schema-Beschreibung erstellen
    schema_info_str = "**Database Schema:**\n"
    for table_name, columns in table_info.items():
        schema_info_str += f"- **Table: {table_name}**\n"
        for column in columns:
            schema_info_str += f"  - {column}\n"

    # Prompt mit Schema anreichern
    modified_prompt = f"{schema_info_str}\n\n**User Query:** {query}"

    try:
        response = db_chain.invoke(modified_prompt)
        return f"{response['result']}"
    except sqlite3.OperationalError as e:
        if "no such table" in str(e):
            return f"Error: Die referenzierte Tabelle existiert nicht. {schema_info_str}"
        elif "no such column" in str(e):
            return f"Error: Die referenzierte Spalte existiert nicht. {schema_info_str}"
        else:
            return f"Fehler bei der Ausführung: {e}"

In [ ]:
def call_chatbot(query: str, chat_history=[]) -> str:
    """Callback-Funktion für die Gradio-Schnittstelle.

    Args:
        query: Benutzeranfrage
        chat_history: Chat-Verlauf (nicht verwendet)

    Returns:
        Antwort des Chatbots
    """
    return run_query_with_schema_check(query)

In [ ]:
# call_chatbot("Which albums are by AC/DC?", [])

# **7 <font color='orange'>|</font> Interaktions- und Testfunktionen**
---

Gradio Interface:

In [ ]:
# Beispielfragen definieren
EXAMPLE_QUESTIONS = [
    "Whitch Albums are from AC/DC?",
    "Who are the artists in Genre Rock?",
    "What was the invoice with the highest total and who was the customer?"
]

# Gradio Interface erstellen
demo = gr.ChatInterface(
    fn=call_chatbot,
    title=APP_TITLE,
    description=APP_DESCRIPTION,
    examples=EXAMPLE_QUESTIONS
)

Gradio App starten:

In [ ]:
demo.launch(share=True)